# Parametric Insurance on the Blockchain

## Description of Scenario

Parametric insurance works by triggering a payment to a client upon the occurence of an event.
This notebook contains a simulation of a parametric insurance application which is hosted on the blockchain.
The demand for such a system is driven by the ability to automate the process of filing and adjusting claims.
This results in lower overhead, which in turn allows insurance firms to reach markets that would otherwise be
cost-prohibitive. The adoption of smart contract based parametric insurance would be mutually beneficial to insurers and their clients.

_'Today, there are over $1 trillion dollars of uninsured crops around the world. To date, insurance companies have largely been unable to service smaller plots of land in developing countries, forcing farmers into unstable livelihoods dependent on rainfall.'_
[Nasdaq: How Blockchain Smart Contracts are Reinventing The Insurance Industry](https://www.nasdaq.com/articles/how-blockchain-smart-contracts-are-reinventing-the-insurance-industry-2021-01-29)


In this particular scenario, we will be outlining the process of insuring a small farm.
Our parametric criteria will be rainfall. 

If a farmer receives a below-threshold amount of rain for a determined
number of consective days, a payout will be issued to the farmer from the insurance firm.

## The Process:

### The Agreement

An insurance firm and a farmer agree upon conditions for insurance. These include;
the monthly cost to the farmer, the payout amount a farmer receives in the event of a drought and the minimum amount of rain a farmer needs to receive per month to avoid a drought payout.

_For the purposes of our demonstration, we will also be able to manipulate the farmer's required dues payperiod, and how frequently a farmer is able to collect drought payouts._

Once the two are in agreement, a client is registered on the insurance firms blockchain contract. The client is given an id number which they will use to pay their dues, and the firm will use to issue payouts.

### Drought Monitoring

A weather station near the farm measures precipitation. Daily rainfall numbers are sent to an oracle. The oracle emits the rainfall amount. If the rainfall does not meet a minimum threshold over a predetermined period of time, a transaction is logged on the blockchain that reports that the drought is occurring.

### Insurance Payouts and Paying Dues

When a drought is reported, if enough time has elapsed since the most recent drought payout, a payout is made to the farmer experiencing the drought. However, the farmer must be in good financial standing with the agency. If the farmer defaults on a payment, they are placed into poor standing with the agency.

A farmer in poor standing may not receive payouts, and they also may not pay dues. The contract is essentially put on hold until further action is taken:

- A client may only reach poor standing by missing a payment, the agency is not able to place them into poor standing.

- If a farmer gets behind on payments, they may contact the agency and rectify their balance outside of the blockchain. At this point the agency is able to change a clients standing from poor to good (but not from good to poor). 

### Transactions

The main contract used in this demonstraction is named Insurance. It is used for tracking Farmer's accounts with the insurance agency. The contract can store multiple clients (farmers). The main purpose of this contract is to collect dues payments from farmers through the function `pay_dues`, and pay farmers in the event of a drought `report_drought`.

### The Code

In order to demonstrate this application, we will be using eth-tester running py-evm as a backend. Py-evm uses a full implementation of the Ethereum spec, allowing us to appropriately mirror our applications behavior if it were to be deployed on the Mainnet.

Execute `Run All`, before proceeding.

In [1]:
from eth_tester import EthereumTester
TESTER = EthereumTester()

In [2]:
# import required testing modules
from solcx import set_solc_version, compile_files
from web3 import Web3

from eth_tester import EthereumTester, PyEVMBackend

import os

/home/austin/miniconda3/envs/kernel/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
# set the solidity version
set_solc_version('v0.7.6')

/home/austin/miniconda3/envs/kernel/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
# create a tester with a PyEVMBackend
TESTER = EthereumTester(backend=PyEVMBackend())

Next we set up accounts for each account user. Our insurance agency will implement our contracts (Insurance.sol, RainOracle.sol), and our farmers will be added to our Insurance concract as clients.

In [5]:
TESTER.get_accounts()

('0x7E5F4552091A69125d5DfCb7b8C2659029395Bdf',
 '0x2B5AD5c4795c026514f8317c7a215E218DcCD6cF',
 '0x6813Eb9362372EEF6200f3b1dbC3f819671cBA69',
 '0x1efF47bc3a10a45D4B230B5d10E37751FE6AA718',
 '0xe1AB8145F7E55DC933d51a18c793F901A3A0b276',
 '0xE57bFE9F44b819898F47BF37E5AF72a0783e1141',
 '0xd41c057fd1c78805AAC12B0A94a405c0461A6FBb',
 '0xF1F6619B38A98d6De0800F1DefC0a6399eB6d30C',
 '0xF7Edc8FA1eCc32967F827C9043FcAe6ba73afA5c',
 '0x4CCeBa2d7D2B4fdcE4304d3e09a1fea9fbEb1528')

In [6]:
insurance_agency = TESTER.get_accounts()[0]
farmer_1 = TESTER.get_accounts()[1]
farmer_2 = TESTER.get_accounts()[2]
farmer_3 = TESTER.get_accounts()[3]

We compile and deploy our insurance contract.

In order to compile and deploy our contracts for testing,
we will be using the following helper functions
(seen first in the SupplyChain Blockchain demonstration by Nodari Gogoberidze).

`get_contract_path`: returns the path the contract given as a parameter

`compile_contract`: compiles the contract with the given path, returns the compiled code.

`get_w3`: takes a provider and returns a web3 object

`deploy_contract`: takes the compiled contract and the w3 as parameters
and deploys the contract to the blockchain

In [7]:
def get_contract_path(contract_name):
    '''
    construct path to contract file
    assumes there is a subdirectory in your current working directory named "contracts"
    
    contract_name: the name of the contract (without the .sol suffix)
    '''
    return os.path.join(os.getcwd(), 'contracts', f'{contract_name}.sol')

def compile_contract(contract_name):
    '''
    compile contract and get the result of compilation
    
    contract_name: the name of the contract, which should match the filename (without the .sol suffix)
    '''
    source_file_name = get_contract_path(contract_name)
    compiled_sol = compile_files([source_file_name]) # Compiled source code
    return compiled_sol[source_file_name + ":" + contract_name]

In [8]:
def get_w3(provider, account=None):
    '''
    get a web3 object instance
    sets the default account to the one provided or the first test account if not provided
    
    provider: backend test provider
    account:  the account number to set as default, or None if you want to use the first test account
    '''
    # web3.py instance
    w3 = Web3(provider)
    
    if account:
        # set account as default sender of transactions
        w3.eth.defaultAccount = account
    else:
        # set the first test account as default sender of transactions
        w3.eth.defaultAccount = w3.eth.accounts[0]
    
    return w3

In [9]:
def deploy_contract(w3, compiled_contract, *c_args):
    '''
    deploy the contract
    
    w3: web3 object instance
    compiled_contract: dictionary of compiled contract should have values for the keys: "abi" and "bin"
    *c_args: a variable number of contract arguments to pass to the constructor of the contract
    '''
    # get the contract interface from the compiled contract
    contract_interface = w3.eth.contract(abi=compiled_contract['abi'], bytecode=compiled_contract['bin'])
    
    # Instantiate the contract by passing in constructor args, and submit the transaction to deploy
    tx_hash = contract_interface.constructor(*c_args).transact()
    
    # Wait for the transaction to be put into a block, and get the transaction receipt
    tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    
    # Get the deployed contract instance from the blockchain with the newly-deployed contract's address
    deployed_contract_interface = w3.eth.contract(
        address=tx_receipt.contractAddress,
        abi=compiled_contract['abi'],
        bytecode=compiled_contract['bin']
    )
    
    return tx_receipt, deployed_contract_interface

### Compiling and Deploying our Main Contract (Insurance.sol)

Our main contract is compiled.

In [10]:
insurance_compiled = compile_contract("Insurance")

We choose a provider, get a W3 object, and deploy our contract. We are using the Ethereum Tester Provider in this demonstration.

In [11]:
provider = Web3.EthereumTesterProvider(ethereum_tester=TESTER)

In [12]:
insurance_w3 = get_w3(provider, insurance_agency)

In [13]:
insurance_receipt, insurance_contract = deploy_contract(insurance_w3, insurance_compiled)

Now that our contract has been compiled, we can view all available functions in the contract.

In [14]:
insurance_contract.all_functions()

[<Function agency_balance()>,
 <Function client_registry(uint256)>,
 <Function contract_owner()>,
 <Function farmer_balance(uint256)>,
 <Function latest_id()>,
 <Function pay_dues(uint256)>,
 <Function pay_farmer(uint256)>,
 <Function rectify_unpaid_dues(uint256)>,
 <Function register_client(uint256,uint256,uint256,uint256,uint256,address)>]

### Compiling and Deploying our Rain Oracle Contract

Our rain oracle contract `RainOracle.sol` will function as a simplified version of data input for our application. In a real world application of this technology an insurance agency would likely want to be processing a variety of weather variables in order to support an insurance claim. 

We will keep it simple for demonstration purposes. Our weather "sensor" will be substituted for manual data input. Our sensor will emit rainful data (rounded to the nearest integer).

We will elaborate on the functionality of this oracle later. For now, the contract is compiled, deployed and we display its functions.

In [15]:
rain_oracle_compiled = compile_contract("RainOracle")

rain_oracle_receipt, rain_oracle_contract = deploy_contract(insurance_w3, rain_oracle_compiled, insurance_agency)

In [16]:
rain_oracle_contract.all_functions()

[<Function owner()>, <Function updateRainfall(uint256,uint256)>]

We will be using three more helper functions (from Nodari's SupplyChain Notebook)

`exec_call`: Issues a call on a function locally, but no transaction is made on the blockchain.

`exec_transact_receipt`: Issues a transaction on the blockchain with the given function. Returns the value returned by the function, and the transaction receipt.

`exec_transact`: Issues a transaction on the blockchain, only returns the value returned by the function.

In [17]:
def exec_call(contract_interface, function_name, *f_args, transaction={}):
    '''
    execute a call, which does not execute a transaction (i.e. no write)
    
    contract_interface: web3 object initialized with compiled contract and its address
    function_name: name of the function in the smart contract to be invoked
    f_args: variable number of arguments to pass in to the function in the smart contract
    transaction: dictionary containing transaction fields
    '''
    func_inst = contract_interface.get_function_by_name(function_name)

    return_value = func_inst(*f_args).call(transaction)
    return return_value

def exec_transact_receipt(contract_interface, function_name, *f_args, transaction={}):
    '''
    execute a transaction (i.e. a write), and return the transaction receipt
    
    contract_interface: web3 object initialized with compiled contract and its address
    function_name: name of the function in the smart contract to be invoked
    f_args: variable number of arguments to pass in to the function in the smart contract
    transaction: dictionary containing transaction fields
    '''
    func_inst = contract_interface.get_function_by_name(function_name)
    
    # get the return value first, without executing transaction
    return_value = exec_call(contract_interface, function_name, *f_args, transaction=transaction)
    
    # execute the transaction
    tx_hash = func_inst(*f_args).transact(transaction)
    # receipt does not contain values returned by function
    tx_receipt = contract_interface.web3.eth.waitForTransactionReceipt(tx_hash)
    
    return return_value, tx_receipt

def exec_transact(contract_interface, function_name, *f_args, transaction={}):
    '''
    execute transaction, but ignore the transaction receipt
    
    contract_interface: web3 object initialized with compiled contract and its address
    function_name: name of the function in the smart contract to be invoked
    f_args: variable number of arguments to pass in to the function in the smart contract
    transaction: dictionary containing transaction fields
    '''
    rv, _ = exec_transact_receipt(contract_interface, function_name, *f_args, transaction=transaction)
    return rv

##### Demonstration: Phase One

In the first phase of this demonstration we will show the process from adding a new farmer, to making a successful payment to the farmer. In the second phase we will look at scenarios where a farmer does not receive a payout.

The contract_owner (the Insurance Agency) is the only party allowed to register new clients. A new client registration takes 6 parameters: 

    - the farmer's monthly cost for the contract (in ether),
    - the payout a farmer receives in the event of a drought (in ether),
    - the minimum number of inches of rain the farmer needs to avoid an insurance payout,
    - the maximum amount of time allowed (in seconds) in between farmer's dues payments,
    - the minimum amount of time (in seconds) in between payouts made to the farmer,
    - the farmer's contract address.

In [18]:
from eth_utils import to_wei, from_wei

In [19]:
# register new farmer
farmer_1_id = exec_transact(insurance_contract, "register_client", 1, 2, 5, 262800, 262800, farmer_1)

In [20]:
# display client object for farmer 1
exec_call(insurance_contract, "client_registry", farmer_1_id)

[0,
 1000000000000000000,
 2000000000000000000,
 5,
 262800,
 262800,
 True,
 0,
 0,
 '0x7E5F4552091A69125d5DfCb7b8C2659029395Bdf',
 '0x2B5AD5c4795c026514f8317c7a215E218DcCD6cF']

- farmer id
- monthly cost to farmer (in wei)
- payout amount to pay to farmer in event of drought (in wei)
- minimum amount of rain in inches required to avoid payout
- maximum time allowed between farmer's dues payments (seconds)
- minimum amount of time allowed between payouts to farmer (seconds)
- payment in good standing
- set last_paid_dues to 0, no dues paid yet
- set last_payout to 0, no payout yet
- agency address
- farmer address

Now that we've added our new farmer to our contract, we will explain how we will process data from our Rain Oracle.

This data transmitted by our oracle will be processed to determine if a farmer is experiencing a drought. A dictionary track each farmer's rainfall-data. 

The dictionary will tally up rainfall for the month. If the rainfall is below the threshold, a drought will be triggered, and a payment will be made to the farmer.

We will start by creating this helper function `minumum_rainfall` which will give us our minimum rainfall for a given farmer based on their id.

In [21]:
def minimum_rainfall(contract, farmer_id):
    farmer_details = exec_call(contract, "client_registry", farmer_id)
    min_inches = farmer_details[3]
    return min_inches

In [22]:
(min_rain) = minimum_rainfall(insurance_contract, farmer_1_id)

print("Farmer %d will receive a payout if they receive less than %d inches of rainfall in a month." % (farmer_1_id, min_rain))

Farmer 0 will receive a payout if they receive less than 5 inches of rainfall in a month.


Our sensor 'detects' rainfall, and when it does it calls the `updateRainfall` function, with the amount of rainfall in inches, and the id of the farmer that is affected as parameters. We save transaction receipts from these events below.

In [23]:
_, rain_receipt_1 = exec_transact_receipt(rain_oracle_contract, "updateRainfall", 1, farmer_1_id)
_, rain_receipt_2 = exec_transact_receipt(rain_oracle_contract, "updateRainfall", 1, farmer_1_id)
_, rain_receipt_3 = exec_transact_receipt(rain_oracle_contract, "updateRainfall", 2, farmer_1_id)
_, rain_receipt_4 = exec_transact_receipt(rain_oracle_contract, "updateRainfall", 4, farmer_1_id)

Our transaction receipt holds a reference to the `RainfallUpdate` event. We write a function below `get_rain_event` to pull the attributes from the event (rainfall and farm_id), passing the rain receipt as a parameter.

In [24]:
def get_rain_event(rain_receipt):
     return rain_oracle_contract.events.RainfallUpdate().processReceipt(rain_receipt)[0]['args']

In [25]:
rain_event_1 = get_rain_event(rain_receipt_1)
rain_event_1

AttributeDict({'rainfall': 1, 'farmer_id': 0})

In [26]:
rain_event_2 = get_rain_event(rain_receipt_2)
rain_event_2

AttributeDict({'rainfall': 1, 'farmer_id': 0})

In [27]:
rain_event_3 = get_rain_event(rain_receipt_3)
rain_event_3

AttributeDict({'rainfall': 2, 'farmer_id': 0})

In [28]:
rain_event_4 = get_rain_event(rain_receipt_4)
rain_event_4

AttributeDict({'rainfall': 4, 'farmer_id': 0})

We define the function, `log_rainfall_event`, which takes in a dictionary. This dictionary has a datetime stamp in the `[farmer_id][0]` place. The `[farmer_id][1]` place is an array of rainfall data.

For each rainfall logged, the current time is compared to the datetime stamp in the dictionary. If 30 days have elapsed (a pay-period), the dictionary entry for the farmer is cleared.

If 30 days have not elapsed since the first rain log, the rainfall amount is added to the array of rainfall data for the farmer.

All rainfall amounts for the farmer in the past pay-period are tallied, if the sum is under the drought threshold, the function returns False. If the sum is over, it returns True.


In [29]:
import datetime
import time

In [30]:
def log_rainfall_event(contract, rain_events, event):
    rain_total = 0
    under_threshold = False
    
    # get the id of the farmer experiencing the rainfall
    farmer_id = event['farmer_id']
    # get the rainfall reported in the event
    new_rainfall = event['rainfall']

    # get the farmer's minimum rainfall to avoid a drought
    farmer_min_rain = minimum_rainfall(contract, farmer_id)
    
    # determine if the new rainfall is over the threshold
    over_threshold = (new_rainfall > farmer_min_rain)
    
    # if the farmer is already contained in the dictionary
    if farmer_id in rain_events:
        if (datetime.datetime.now() - rain_events[farmer_id][0]).seconds > 259200:
#           if the rain event occurred more than a month after
#           the first rain in the dictionary, make the current rain event
#           the new first event for this farmer, and wipe rainfall data for the prior month
            rain_events[farmer_id] = [datetime.datetime.now(), [0, new_rainfall]]
        else:
#           if the rain event ocurred within a month of the
#           first rain in the dictionary, add the rain event to the dictionary
            rain_events[farmer_id][1].append(new_rainfall)
#           check if the total rainfall for this farmer is over the drought threshold
#           if so, set over_threshold to True
            for number in rain_events[farmer_id][1]:
                rain_total += number
            if rain_total > farmer_min_rain:
                over_threshold = True
    else:
        # if the farmer is not contained in the dictionary, add the farmer
        rain_events[farmer_id] = [datetime.datetime.now(), [0, new_rainfall]]
            
        
    return over_threshold

In [31]:
from pprint import pprint

In [32]:
# initialize dictionary for logging raindall events
rain_events = dict()

In [33]:
# print the drought threshold for a given farmer
def drought_threshold(farmer_id):
    print("%s inches maximum for payment" % exec_call(insurance_contract, "client_registry", farmer_id)[3])

In [34]:
# print the cumulative rain inches logged in a given period
def total_rainfall(farmer_id):
    cumulative = 0
    for i in rain_events[farmer_id][1]:
        cumulative += i
    print("%s inches total rainfall" % cumulative)

In [35]:
# checks if rainfall is under the drought threshold, if so pay_farmer function is executed.
def pay_farmer(over_threshold, farmer_id, value):
    if not over_threshold:
        exec_transact(insurance_contract, "pay_farmer", farmer_id, transaction={'value' : value})
        print("payment made to farmer %s" %farmer_id)
    else:
        print("Rainfall over drought threshold.")

In [36]:
# calls contract which pays farmer's dues
def pay_dues(farmer_id, farmer, value):
    exec_transact(insurance_contract, "pay_dues", farmer_id, transaction={'from' : farmer, 'value' : value})

In [37]:
# print farmer and agency account balances
def balances(farmer_id):
    print("farmer balance: %s" % from_wei(exec_transact(insurance_contract, "farmer_balance", farmer_id),'ether'))
    print("agency balance: %s" % from_wei(exec_transact(insurance_contract, "agency_balance"),'ether'))

### Drought

Below we show the functionality of our contract in the event that a farmer is experiencing a drought. Rain events 1-3 total to 4 inches of rain. Our 4th rain event puts us over the threshold for a drought payment, so for this demonstraction we will only log events 1-3.

We can imagine that these events occur over a 4 week period. The regularly attempts to issue a payment at the end of the month. Because the farmer's threshold of 5 inches of rain has not been reached, the contract will issue a payment to the farmer. 

In [38]:
threshold_1 = log_rainfall_event(insurance_contract, rain_events, rain_event_1)
threshold_2 = log_rainfall_event(insurance_contract, rain_events, rain_event_2)
threshold_3 = log_rainfall_event(insurance_contract, rain_events, rain_event_3)

drought_threshold(farmer_1_id)
total_rainfall(farmer_1_id)

5 inches maximum for payment
4 inches total rainfall


In [39]:
balances(farmer_1_id)

farmer balance: 1000000
agency balance: 999999.999999999997992287


In [40]:
pay_farmer(threshold_3, farmer_1_id, to_wei(2, 'ether'))

payment made to farmer 0


In [41]:
balances(farmer_1_id)

farmer balance: 1000002
agency balance: 999997.999999999997894801


## Demonstration: Phase Two

In this second phase we will look at scenarios where a farmer does not receive a payout. This would be one of two scenarios. Either A) B) A drought is triggered but the farmer has already been paid this payperiod (1 month). B) No drought is not trigged, or C) A farmer has not paid their dues.

### Scenario A : Double Payment

A farmer may only receive a payout once every thirty days. If a drought is triggered again within this payperiod, the farmer will not be paid.

In [42]:
try:
    pay_farmer(threshold_3, farmer_1_id, to_wei(2, 'ether'))
except:
    print("Last payout was too recent, try again next pay-period.")

Last payout was too recent, try again next pay-period.


### Scenario B : No Drought

We build on our previous sensor data. We now log rainfall event 4 in addition to 1-3. We will now pretend that we are in a new month. All four of these rain events have occured in this new month. Because our cumulative rainfall is now at 8 inches for this month, we are over our 5 inch threshold, and our drought payment is not completed.

In [43]:
threshold_4 = log_rainfall_event(insurance_contract, rain_events, rain_event_4)

drought_threshold(farmer_1_id)
total_rainfall(farmer_1_id)

5 inches maximum for payment
8 inches total rainfall


In [44]:
pay_farmer(threshold_4, farmer_1_id, to_wei(2, 'ether'))

Rainfall over drought threshold.


### Scenario C : Dues Unpaid

In the event that a farmer has not paid their dues for the payperiod, the farmer will be unable to make or receive payments.

For this scenario we will initialize a new client. Farmer 2 will have a very short payperiod. Meaning, if they go several seconds without paying their dues, they will default on their payments (be placed in bad standing with the agency). We will make an initial dues payment (required to start keeping track of dues payment timestamps), and then we will attempt to issue a drought to the farmer. At this point, the farmer will have missed a dues payment, and therefore they will be unable to receive an insurance payment.

In [45]:
farmer_2_id = exec_transact(insurance_contract, "register_client", 1, 2, 5, 2, 200000, farmer_2)

In [46]:
_, rain_receipt_5 = exec_transact_receipt(rain_oracle_contract, "updateRainfall", 3, farmer_2_id)

In [47]:
rain_event_5 = get_rain_event(rain_receipt_5)
rain_event_5

AttributeDict({'rainfall': 3, 'farmer_id': 1})

In [48]:
# log rainfall event, print drought threshold and total rainfall this period
threshold_5 = log_rainfall_event(insurance_contract, rain_events, rain_event_5)

drought_threshold(farmer_2_id)
total_rainfall(farmer_2_id)

5 inches maximum for payment
3 inches total rainfall


In [49]:
balances(farmer_2_id)

farmer balance: 1000000
agency balance: 999997.999999999997614607


In [50]:
# pay dues to initialize payment cycle timer (set client.last_paid_dues = current blocktime)
pay_dues(farmer_2_id, farmer_2, to_wei(1, 'ether'))

In [51]:
balances(farmer_2_id)

farmer balance: 999998.999999999999947372
agency balance: 999998.999999999997570025


In [52]:
# trigger payout for farmer_2, dues payment period should have elapsed at this point
try:
    pay_farmer(threshold_5, farmer_2_id, to_wei(2, 'ether'))
except:
    print("Too much time has elapsed since last dues paid, contact agency to rectify balance owed.")

Too much time has elapsed since last dues paid, contact agency to rectify balance owed.


### Continued : Dues Unpaid

In the event that a farmer finds themselves in poor financial standing with the insurance agency (has missed a payment), there is a function for rectifying this situation.

A farmer in poor standing will be unable to receive insurance payouts.

If the agency is able to rectify the charges owed to them (recieve payment outside of the blockchain). The function rectify_unpaid_dues allows the Insurance Agency to place the farmer in question back into good standing, thus re-establishing their ability to receive payouts during a drought.

In [53]:
exec_transact(insurance_contract, "rectify_unpaid_dues", farmer_2_id, transaction={'from' : insurance_agency})

'unpaid dues rectified'

In [54]:
balances(farmer_2_id)

farmer balance: 999998.999999999999947372
agency balance: 999998.999999999997511402


The following code must run sequentially. We do not check our balance after paying dues because we have set the payperiod for this farmer to be extremely short. If we check the farmer's balance after paying dues, blocktime will advance and our farmer will once again be placed into poor standing, and will be unable to recieve a payout.

In [55]:
pay_dues(farmer_2_id, farmer_2, to_wei(1, 'ether'))

In [56]:
try:
    pay_farmer(threshold_5, farmer_2_id, to_wei(2, 'ether'))
except:
    print("Too much time has elapsed since last dues paid, contact agency to rectify balance owed.")

payment made to farmer 1


In [57]:
balances(farmer_2_id)

farmer balance: 999999.999999999999894744
agency balance: 999997.999999999997414027
